## Algoritmo 
(se considera que los índices de los arrays inician en el valor $1$ y no en el $0$)

Entrada: $A \in \mathbb{R}^{n \times n}$.

Salida: factores $L, U \in \mathbb{R}^{n \times n}$, vector $piv \in \mathbb{R}^{n}$. El vector $piv$ se utiliza para construir a la matriz $P$.

Inicializar L con matriz identidad. Inicializar U como una matriz de ceros.

Para $j$ de $1$ a $n$ hacer:

Si $j$ es 1 definir $v = A(:,1)$ # v es la primer columna de $A$.

En otro caso:

Calcular $\hat{a} = P_{j-1} \cdots P_1 A(:,j)$ #aquí se utiliza al vector $piv$ para evitar hacer multiplicaciones de matriz vector, sólo se realizan intercambio de renglones

Resolver $L(1:j-1,1:j-1)z = \hat{a}(1:j-1)$ para $z \in \mathbb{R}^{j-1}$ por sustitución hacia delante

Definir $U(1:j-1,j) = z$

Definir $v(j:n) = \hat{a}(j:n) - L(j:n, 1:j-1)*z$

Determinar $\mu$ con $ j \leq \mu \leq n$ tal que $|v(\mu)| = ||v(j:n)||_\infty$

Definir $piv(j)=\mu$ # piv es un vector de tamaño $n-1$

$v(j) \leftrightarrow v(\mu)$ # $\leftrightarrow$ significa intercambiar $j$-ésima entrada de $v$ por $\mu$-ésima entrada de $v$.

$L(j,1:j-1) \leftrightarrow L(\mu,1:j-1)$ #$\leftrightarrow$ significa intercambio

Definir $U(j,j) = v(j)$.

Si $v(j) \neq 0 $ definir $L(j+1:n,j) = v(j+1:n)/v(j)$


In [1]:
import numpy as np

In [10]:
n = 3
A = np.array([[ 3.0, -1.0,4.0], \
        [-2.0, 0.0, 5.0],
        [ 7.0, 2.0, -2.0]])

#Inicializar L con matriz identidad. Inicializar U como una matriz de ceros.
L = np.identity(n)
U = np.zeros((n,n))


A[:,0]
np.eye(3)
len(A)

3

In [ ]:
#Entrada:  𝐴∈ℝ𝑛×𝑛 .
#Salida: factores  𝐿,𝑈∈ℝ𝑛×𝑛 , vector  𝑝𝑖𝑣∈ℝ𝑛 . El vector  𝑝𝑖𝑣  se utiliza para construir a la matriz  𝑃 .


"""
 Parameters
    ----------
   n : int
       matrix dimension
"""


A = np.array([[ 3.0, -1.0,4.0], \
        [-2.0, 0.0, 5.0],
        [ 7.0, 2.0, -2.0]])

n = len(A)

#Inicializar L con matriz identidad. Inicializar U como una matriz de ceros.
L = np.identity(n)
U = np.zeros((n,n))

P =  np.identity(n)
#Para  𝑗  de  1  a  𝑛  hacer:
for j in 1:n:
    #Si  𝑗  es 1 definir  𝑣=𝐴(:,1)  # v es la primer columna de  𝐴 .
    if j ==1 :
        v = A[:,0] #Python empieza en 0
    #En otro caso:
    else:
        #Calcular  𝑎̂ =𝑃𝑗−1⋯𝑃1𝐴(:,𝑗)  #aquí se utiliza al vector  𝑝𝑖𝑣  para evitar hacer multiplicaciones de matriz vector, sólo se realizan intercambio de renglones
        a_hat = #NO ENTIENDO
    
    #Resolver  𝐿(1:𝑗−1,1:𝑗−1)𝑧=𝑎̂ (1:𝑗−1)  para  𝑧∈ℝ𝑗−1  por sustitución hacia delante

    
Definir  𝑈(1:𝑗−1,𝑗)=𝑧 
Definir  𝑣(𝑗:𝑛)=𝑎̂ (𝑗:𝑛)−𝐿(𝑗:𝑛,1:𝑗−1)∗𝑧 
Determinar  𝜇  con  𝑗≤𝜇≤𝑛  tal que  |𝑣(𝜇)|=||𝑣(𝑗:𝑛)||∞ 
Definir  𝑝𝑖𝑣(𝑗)=𝜇  # piv es un vector de tamaño  𝑛−1 
𝑣(𝑗)↔𝑣(𝜇)  #  ↔  significa intercambiar  𝑗 -ésima entrada de  𝑣  por  𝜇 -ésima entrada de  𝑣 .

𝐿(𝑗,1:𝑗−1)↔𝐿(𝜇,1:𝑗−1)  # ↔  significa intercambio

Definir  𝑈(𝑗,𝑗)=𝑣(𝑗) .

Si  𝑣(𝑗)≠0  definir  𝐿(𝑗+1:𝑛,𝑗)=𝑣(𝑗+1:𝑛)/𝑣(𝑗)

### ¿Dado el sistema $Ax=b$, $A \in \mathbb{R}^{n \times n}$ cómo se resuelve con la factorización $PLU$?
Paso 1: encontrar factores $P,L,U$ tales que $PA=LU$.

Paso 2: resolver con el método de sustitución hacia delante el sistema triangular inferior $Ld=Pb$.

Paso 3: resolver con el método de sustitución hacia atrás el sistema triangular superior $Ux=d$.